# JUICE RPWI HF SID5 (PSSR1 surv): L1a QL -- 2025/7/5

# Import lib

In [ ]:
import copy
import datetime
import math
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import sys

# CDF and Directory setting: set by User

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf
import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_1-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cal_lib   as juice_cal
import juice_cdf_lib   as juice_cdf
import juice_math_lib  as juice_math
import juice_sid5_data as juice_data
import juice_sid5_lib  as juice_sid5
import juice_spec_lib  as juice_spec

# Setting and Read CDF file: set by User

In [ ]:
# *** Dump ***
dump_mode = 2                           # 0: no-dump  1:plot dump
# *** CAL ***
unit_mode = 2                           # [Power]     0: raw     1: V＠ADC     2: V@HF    3: V@RWI  4: V/m
band_mode = 0                           # [Power]     0: sum     1: /Hz
cal_mode  = 2                           # [Power]     0: background     1: CAL     2: all
# *** Unit ***
f_mode    = 1                           # [Frequency] 0: linear  1: log
time_mode = 1                           # [Time]      0: Epoch   1: data number
gap_mode  = 1                           # [gap]       0: n/a     1: gap
# *** Power range ***
p_raw_max = 8.5                         # background: 7.5   CAL: 10
p_raw_min = 1.0                         # background: 2.5   CAL: 5
# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder

In [ ]:
E_min = '2024-08-19 20:30:00';  t_min0 = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S")
E_max = '2024-08-19 20:40:00';  t_max0 = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S")
t_min0 = 0;  t_max0 = 0

# get CDF data

In [ ]:
# date='20240909'
# date='20250331'; ver = 'V02'
date='0'; ver = 'V02'
data_dir, data_name_list = juice_data.datalist(date, ver)       # [date]   yyyymmdd: group read    others: file list

In [ ]:
class struct:
    pass

data = struct()
num_list = len(data_name_list)
for i in range(num_list):
    data_name = data_name_list[i];  cdf_file = data_dir + data_name;  print(i, cdf_file)
    RPWI_FSW_version = juice_cdf._RPWI_FSW_version(cdf_file)
    cdf = pycdf.CDF(cdf_file);      data1 = juice_sid5.hf_sid5_read(cdf, RPWI_FSW_version)
    if i==0:
        data = data1;                               print(data.EE.shape)
    else:
        data = juice_sid5.hf_sid5_add(data, data1); print(data.EE.shape)
data_name = os.path.split(data_name)[1];            print("data name:", data_name)

In [ ]:
data = juice_sid5.hf_sid5_shaping(data, cal_mode)

In [ ]:
date1 = data.epoch[0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]; date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2
n_time0 = data.EE.shape[0];     n_freq0 = data.EE.shape[1]
f_min0  = data.frequency[0][0]; f_max0  = (max(np.ravel(data.frequency)))

print("       date and time:", str_date)
print("           data size:", data.EE.shape, "  [ Num-samples:", n_time0, "  Num-Frequency:", n_freq0, "]")
print("           Frequency:", f_min0,  "-", f_max0, "kHz")
print("f, width, step (kHz):", f_min0,  "-", f_max0, data.freq_width[0][0], data.freq_step[0][0])

# N_ch0     = data.U_selected[0] + data.V_selected[0] + data.W_selected[0]
print("                  Ch:", data.ch_selected[0])
#
N_frequency = data.n_step 
N_component = 1;    print("       ==> Component:", N_component)

# Spectrum Data

In [ ]:
T_HF  = data.T_HF_FPGA[n_time0//2];  T_RWI = (data.T_RWI_CH1[n_time0//2] + data.T_RWI_CH2[n_time0//2])/2
print("Temperature(org): ", 'HF - {:.1f}'.format(T_HF), '  RWI - {:.1f}'.format(T_RWI))
if T_HF  > 199 or T_HF  < -50 or math.isnan(T_HF):
   T_HF  = 25;  data.T_HF_FPGA[:] = math.nan
if T_RWI > 199 or T_RWI < -199 or math.isnan(T_RWI):
   T_RWI = 25;  data.T_RWI_CH1[:] = math.nan;  data.T_RWI_CH2[:] = math.nan
print("Temperature(cal): ", 'HF - {:.1f}'.format(T_HF), '  RWI - {:.1f}'.format(T_RWI))

In [ ]:
spec = copy.deepcopy(data)
spec = juice_cal.spec_cal(spec,   5, unit_mode, band_mode, T_HF, T_RWI)     # CAL

Epoch_1d  = spec.epoch.tolist();      n_time1 = spec.EE.shape[0];   num_1d = np.arange(n_time1)
freq_1d   = spec.freq  [n_time1//2];  n_freq1 = freq_1d.shape[0];   f_min0 = freq_1d[0];  f_max0  = freq_1d[-1]
freq_w_1d = spec.freq_w[n_time1//2]

# NAN
if gap_mode == 1 and time_mode == 0:
    for i in range(n_time1-1):
        if spec.scet[i+1]-spec.scet[i] > 60:
            print("[gap]", i, spec.epoch[i], i+1, spec.epoch[i+1])
            juice_sid5.spec_nan(spec, i);  juice_sid5.spec_nan(spec, i+1)

In [ ]:
p_max0 = p_raw_max + spec.cf/10;  p_min0 = p_raw_min + spec.cf/10
p_max0 = np.ceil(np.log10( np.nanmax(spec.EE) )*5)/5+.5
p_min0 = np.ceil(np.log10( np.nanmin(spec.EE) )*5)/5-.5
p_max  = p_max0;                  p_min  = p_min0
print("conversion factor:", '{:.1f}'.format(spec.cf), "    MAX-min:", '{:.1f}'.format(p_max), '({:.1f})'.format(p_max0), '{:.1f}'.format(p_min), '({:.1f})'.format(p_min0), "   str_unit:", spec.str_unit)

### Peak

In [ ]:
# print("N_samp:", data.N_samp[0], "   decimation:", juice_cdf._sample_rate(data.decimation[0]))
peak_E = np.ravel(spec.EE[:][:]); peak_f = np.nanargmax(peak_E)
print("Peak EE:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), '  |  {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])), spec.str_unit, ' {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])**.5))
if unit_mode==0:
    peak_E = np.ravel(spec.EE[:][:]); peak_f = np.nanargmax(peak_E);  print("Peak EuEu:", '{:10.0f}'.format(peak_E[peak_f]), '{:x}'.format(np.int64(peak_E[peak_f])))

In [ ]:
n_sweep = n_time1//2;  peak_E = np.ravel(spec.EE[n_sweep]);  peak_f = np.nanargmax(peak_E); 
print(f"[SWEEP - {n_sweep}]  Peak at", '{:.1f}'.format(freq_1d[peak_f]), "kHz", peak_f)
print("     EE:", '{:+.2e}'.format(np.ravel(spec.EE   [n_sweep])[peak_f]), spec.str_unit)

In [ ]:
n_sweep1 = 0;  n_sweep2 = n_time0//2;  n_sweep3 = n_time0-1
peak_E = np.ravel(spec.EE[n_sweep1]); peak_f = np.nanargmax(peak_E); print(f"[First peak - {n_sweep1}]  EE:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EE[n_sweep2]); peak_f = np.nanargmax(peak_E); print(f"[Mid   peak - {n_sweep2}]  EE:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
peak_E = np.ravel(spec.EE[n_sweep3]); peak_f = np.nanargmax(peak_E); print(f"[Last  peak - {n_sweep3}]  EE:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))

### Spectrum plot

In [ ]:
fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(4, 1, 1);  ax7 = fig.add_subplot(4, 1, 2);  ax8 = fig.add_subplot(4, 1, 3);  ax9 = fig.add_subplot(4, 1, 4)

ax1.plot(np.ravel(spec.EE),             '-r', linewidth=1.2, label='EE')
ax1.set_yscale('log')
ax7.plot(np.ravel(data.frequency),      '-r', linewidth=0.6, label='frequency')
ax7.plot(np.ravel(data.freq_step*10),   '-g', linewidth=0.8, label='step*10')
ax7.plot(np.ravel(data.freq_width*10),  ':b', linewidth=1.0, label='width*10')
ax8.plot(np.ravel(data.T_HF_FPGA),      ':r', label='T (HK-FPGA)')
ax8.plot(np.ravel(data.T_RWI_CH1),      ':b', label='T (RWI1)')
ax8.plot(np.ravel(data.T_RWI_CH2),      ':g', label='T (RWI2)')
ax8.plot(np.ravel(data.cal_signal*10),  '-k', label='CAL')
ax9.plot(np.ravel(data.epoch),          '.')

ax1.set_ylabel(spec.str_unit)
ax7.set_ylabel('frequency [kHz]');    ax8.set_ylabel('T [degC]');   ax9.set_ylabel('Date-Time');  ax9.set_xlabel(str_date)
#
title_label = '[JUICE/RPWI HF PSSR1-Surv (SID-5)]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);     ax7.legend(loc='upper right', fontsize=8); ax8.legend(loc='upper right', fontsize=8)

xlim=[-.5, len(np.ravel(data.EE))   -.5]; print(xlim); ax1.set_xlim(xlim); ax7.set_xlim(xlim)
xlim=[-.5, len(np.ravel(data.epoch))-.5]; print(xlim); ax8.set_xlim(xlim); ax9.set_xlim(xlim)
# ylim=[10**p_min, 10**p_max];                ax1.set_ylim(ylim)
ylim=[f_min0, f_max0];  ax7.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

In [ ]:
n_time = n_time1//2
fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(2, 1, 1);  ax7 = fig.add_subplot(2, 1, 2)

ax1.plot(spec.EE[n_time],            '-r', linewidth=1.2, label='EE')
ax1.set_yscale('log')
ax7.plot(data.frequency[n_time],     '-r', linewidth=0.6, label='frequency')
ax7.plot(data.freq_step[n_time]*10,  '-g', linewidth=0.8, label='step*10')
ax7.plot(data.freq_width[n_time]*10, ':b', linewidth=1.0, label='width*10')

ax1.set_ylabel(spec.str_unit)
ax7.set_ylabel('frequency [kHz]');    ax7.set_xlabel(data.epoch[n_time].strftime('%Y/%m/%d %R:%S') + '  [in ' + str_date + ']')
#
title_label = '[JUICE/RPWI HF PSSR1-Rich (SID-21)]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);      ax7.legend(loc='upper right', fontsize=8)

xlim=[0, n_freq0];  ax1.set_xlim(xlim); ax7.set_xlim(xlim)
# ylim=[10**p_raw_min, 10**p_raw_max]; # ax1.set_ylim(ylim)
ylim=[f_min0, f_max0];  ax7.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw2.png'
    fig.savefig(png_fname)

In [ ]:
print("CAL_sig:", data.cal_signal)

### FT Plot data

In [ ]:
# Normal spectrum plots
EE_2d      = spec.EE.transpose();       EE_med      = np.nanmedian(spec.EE,      axis=0)
f_min0 = freq_1d[0];  f_max0 = freq_1d[N_frequency-1]
print(spec.str_unit, "   f_MAX/min:", f_max0, f_min0, "   num of f, step, window:", N_frequency, data.freq_step[0][0], data.freq_width[0][0])

In [ ]:
"""
Epoch_1d      = data.epoch.tolist()
freq_1d       = data.frequency[0].tolist()
freq1         = freq_1d[0:N_frequency]
freq_width_1d = data.freq_width[0].tolist()
freq_width1   = freq_width_1d[0:N_frequency]

EE_2d = data.EE.transpose()
# EE_power1 = data.EE[0][0:N_frequency]
# EE_power2 = data.EE[n_time0][0:N_frequency]
#
freq_width_2d = data.freq_width.transpose()
if band_mode == 1:
    EE_2d = EE_2d / freq_width_2d / 1000 * data.subdiv_reduction[0]
    # EE_power1 = EE_power1 / freq_width1 / 1000 * data.subdiv_reduction[0]
    # EE_power2 = EE_power2 / freq_width1 / 1000 * data.subdiv_reduction[0]
f_min0 = freq_1d[0]
f_max0 = freq_1d[N_frequency-1]
# power_str = juice_cdf.power_label(band_mode, unit_mode)
"""

In [ ]:
n0 = 0;  n1 = n_time0//2;   n2 = n_time0-1
p_min = p_min0;  p_max = p_max0
f_min = f_min0;  f_max = f_max0

fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(1, 1, 1)

ax1.plot(freq_1d, EE_med,      '-y',  linewidth=3.0, label='med')
ax1.plot(freq_1d, spec.EE[n0], ':r',  linewidth=1.0, label='init')
ax1.plot(freq_1d, spec.EE[n1], '-.r', linewidth=1.0, label='half')
ax1.plot(freq_1d, spec.EE[n2], '-r',  linewidth=1.0, label='end')
ax1.set_yscale('log')
if f_mode == 1: ax1.set_xscale('log')

ax1.set_ylabel(spec.str_unit)
ax1.set_xlabel('frequency [Hz]')

date1 = data.epoch[n0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[n1];  date2 = date2.strftime('%Y/%m/%d %R:%S')
date3 = data.epoch[n2];  date3 = date3.strftime('%Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1 + "  -  " + date2 + "  -  " + date3;  ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8)

xlim=[f_min, f_max];  ax1.set_xlim(xlim)
ylim=[10**p_min, 10**p_max]; ax1.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode > 0:
    png_fname = work_dir+data_name+'_spec.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-log.png'
    fig.savefig(png_fname)

## FT

In [ ]:
# Y-range
f_min = f_min0;      f_max = f_max0
# Z-range
p_min = p_min0;      p_max = p_max0

fig2d = plt.figure(figsize=[16,11])
ax1 = fig2d.add_subplot(1, 1, 1)

# Y-axis
ax1.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title("[" + data_name + "]  " + 'EE')
ax1.set_xlabel(str_date)

# Plot
p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EE_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')

pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(spec.str_unit)

plt.show()
if dump_mode > 0:
    png_fname = work_dir+data_name+'_FT.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-log.png'
    fig2d.savefig(png_fname)
